In [46]:
%load_ext autoreload
%autoreload 2
import os
import sys

from config_ai.utils import *
from ai_schema import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import glob
from jsonpath_ng import parse
import pandas as pd


def analyse_project(project, metric_path="$.micro.f1"):
    project_path = f"/nfs/pony/chenhao/experiment/{project}"
    expected_expr = parse(metric_path)
    rs_list = []

    for model in os.listdir(project_path):
        if model == "tensorboard":
            continue
#         print(model)
        path = os.path.join(project_path, model)     
        for tag in ["train", "dev"]:
            metrics = []
            evals = []
            max_value=0
            max_eval=None
            max_dir = None
            for filename in glob.glob(f"{path}/**/eval/{tag}.json") + glob.glob(f"{path}/eval/{tag}.json"):
                print(filename)
                eval_rs = jload(filename)
                metric = [a.value for a in expected_expr.find(eval_rs)][0]
                metrics.append(metric)
                evals.append(eval_rs)
                
                if metric > max_value:
                    max_value=metric
                    max_eval = eval_rs
                    max_dir = filename[:filename.index("eval")]
            avg_metric = sum(metrics)/len(metrics) if metrics else 0
            rs_list.append(dict(project=project, model=model, tag=tag, avg_metric=avg_metric, metric=max_value, path=max_dir, detail=max_eval))
    df = pd.DataFrame.from_records(rs_list)
    return df

            

            
    


In [48]:
# eval_df = analyse_project("config_ai_warmup")
# eval_df = analyse_project("clue_ocnli")
eval_df = analyse_project("clue_wsc")

/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:30:00/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:30:07/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:32:10/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:30:00/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:30:07/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr2/2021-08-13-11:32:10/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr3/2021-08-13-11:32:10/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr3/2021-08-13-11:36:09/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr3/2021-08-13-11:37:07/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_nowarm_lr3/2021-08-13-1

/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-15:14:32/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-15:16:32/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-17:13:32/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-17:13:33/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-17:13:34/eval/train.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-15:14:26/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-15:14:32/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-15:16:32/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_drop2_nordrop/2021-08-13-17:13:32/eval/dev.json
/nfs/pony/chenhao/experiment/clue_wsc/relation_classif

In [49]:
dev_df = eval_df[eval_df["tag"] == "dev"].sort_values(["avg_metric"], ascending=False)
dev_df.head(20)
dev_df["path"].tolist()[:5]

dev_df["detail"].to_list()

,project,model,tag,avg_metric,metric,path,detail
53,clue_wsc,relation_classify_large_fold-1,dev,0.925806,0.925806,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 184, 'fp': 12, 'fn..."
47,clue_wsc,relation_classify_fold-1,dev,0.916129,0.916129,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 181, 'fp': 12, 'fn..."
45,clue_wsc,relation_classify_fold-3,dev,0.912621,0.912621,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 168, 'fp': 19, 'fn..."
61,clue_wsc,relation_classify_large_fold-3,dev,0.902913,0.902913,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 171, 'fp': 25, 'fn..."
59,clue_wsc,relation_classify_fold-4,dev,0.899676,0.899676,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 175, 'fp': 21, 'fn..."
63,clue_wsc,relation_classify_large_fold-4,dev,0.896440,0.896440,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 175, 'fp': 22, 'fn..."
55,clue_wsc,relation_classify_large_fold-0,dev,0.893548,0.893548,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 169, 'fp': 23, 'fn..."
51,clue_wsc,relation_classify_fold-2,dev,0.887097,0.887097,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 165, 'fp': 15, 'fn..."
49,clue_wsc,relation_classify_fold-0,dev,0.887097,0.887097,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 170, 'fp': 26, 'fn..."
13,clue_wsc,relation_classify_ENTITY_START_END,dev,0.851974,0.875000,/nfs/pony/chenhao/experiment/clue_wsc/relation...,"{'detail': {'false': {'tp': 176, 'fp': 21, 'fn..."


['/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_large_fold-1/2021-08-16-10:42:39/',
 '/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_fold-1/2021-08-16-10:39:09/',
 '/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_fold-3/2021-08-16-10:38:11/',
 '/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_large_fold-3/2021-08-16-10:58:40/',
 '/nfs/pony/chenhao/experiment/clue_wsc/relation_classify_fold-4/2021-08-16-10:54:36/']

[{'detail': {'false': {'tp': 184,
    'fp': 12,
    'fn': 11,
    'precision': 0.9387755102040817,
    'recall': 0.9435897435897436,
    'f1': 0.9411764705882353},
   'true': {'tp': 103,
    'fp': 11,
    'fn': 12,
    'precision': 0.9035087719298246,
    'recall': 0.8956521739130435,
    'f1': 0.8995633187772927}},
  'micro': {'tp': 287,
   'fp': 23,
   'fn': 23,
   'precision': 0.9258064516129032,
   'recall': 0.9258064516129032,
   'f1': 0.9258064516129032},
  'macro': {'precision': 0.9211421410669531,
   'recall': 0.9196209587513935,
   'f1': 0.9203809213666854}},
 {'detail': {'false': {'tp': 181,
    'fp': 12,
    'fn': 14,
    'precision': 0.9378238341968912,
    'recall': 0.9282051282051282,
    'f1': 0.9329896907216495},
   'true': {'tp': 103,
    'fp': 14,
    'fn': 12,
    'precision': 0.8803418803418803,
    'recall': 0.8956521739130435,
    'f1': 0.8879310344827586}},
  'micro': {'tp': 284,
   'fp': 26,
   'fn': 26,
   'precision': 0.9161290322580645,
   'recall': 0.9161290

In [21]:
tmp[:2]

[{'text': '怎么我的电费也是花呗缴费呀',
  'extra_text': '电费代扣成功花呗怎么查不到',
  'label': {'name': '0', 'prob': 1.0},
  'predict': {'name': '0', 'prob': 0.6011787056922913},
  'tp_set': ['0'],
  'fp_set': [],
  'fn_set': []},
 {'text': '网商贷借呗转换',
  'extra_text': '不想要网商贷能不能切换回借呗',
  'label': {'name': '1', 'prob': 1.0},
  'predict': {'name': '0', 'prob': 0.652289628982544},
  'tp_set': [],
  'fp_set': ['0'],
  'fn_set': ['1']}]